In [ ]:
20

In [ ]:
20

In [1]:
from federatedscope.register import register_data
from federatedscope.register import register_trainer
from federatedscope.register import register_metric
from federatedscope.register import register_model




### Register model

In [2]:
#from federatedscope.contrib.model.mnist_model import call_my_net
#register_model("mynet", call_my_net)

### Register trainer

In [3]:


from federatedscope.contrib.trainer.laplacian_trainer_with_domain_separation_with_summation_1MINE_VAE import call_laplacian_trainer

register_trainer('laplacian_trainer', call_laplacian_trainer)

/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "
/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch_geometric/graphgym/logger.py:23: UserWarning: Please install 'pytorch_lightning' for using the GraphGym experiment manager via 'pip install pytorch_lightning'
  warnings.warn("Please install 'pytorch_lightning' for using the GraphGym "


Device: cuda


### Register metric

### Create the config

#### Set data, model, trainer and metric

In [4]:
from federatedscope.core.configs.config import global_cfg, CN
cfg = global_cfg.clone()

cfg.merge_from_file("scripts/B-FHTL_exp_scripts/Graph-DT/fed_dom_sep_MI.yaml")
cfg.data.save_dir = 'test_dir'
# cfg.data.type = 'cikm_cup'
#cfg.data.root = 'data'
#cfg.data.shuffle=True
#cfg.data.transform = [['ToTensor'], ['Normalize', {'mean': [0.], 'std': [1]}]]
#cfg.model.type = 'gin'
#cfg.model.out_channels = 10
#cfg.model.hidden = 64
#cfg.model.task='graph'
#cfg.model.dropout = 0.5
#cfg.personalization.local_param = ['encoder_atom', 'encoder', 'clf']#['node_encoder', 'clf']
#cfg.train.batch_or_epoch = "epoch"
cfg.trainer.type = 'laplacian_trainer'
cfg.data.batch_size = 64
# cfg.eval.metric = ['mymetric']
cfg.params = CN()
cfg.params.alpha=0.1
cfg.params.csd_importance= 1e2
cfg.params.diff_importance = 1
cfg.params.mine_lr = 0.01
cfg.params.eps=1e-20
cfg.params.p=0.
cfg.params.lam = 0.01


#### configure other options

In [5]:
#cfg.use_gpu = True 0.05 [0.02-0.03] 0.04-0.06
#cfg.best_res_update_round_wise_key = "test_loss"

#cfg.federate.mode = 'standalone'

cfg.federate.method = \
    f'Laplacian_MINE_VAE_test'

#cfg.federate.local_update_steps = 20000000
cfg.personalization.local_update_steps = 20000000
#cfg.finetune.local_update_steps = 20000000
#cfg.train.local_update_steps = 1

cfg.federate.total_round_num = 2000
cfg.federate.client_num = 3
cfg.early_stop.patience = 3000
#cfg.train.optimizer.lr = 0.001
#cfg.train.optimizer.weight_decay = 0.0005
#cfg.grad.grad_clip = 2.0
cfg.criterion.type = 'CrossEntropyLoss'
#cfg.seed = 123
cfg.eval.freq = 1
cfg.eval.metrics = ['imp_ratio']
cfg.eval.report = ['avg']
cfg.eval.best_res_update_round_wise_key = 'val_imp_ratio'
cfg.eval.count_flops = False


In [6]:
import torch
#torch.manual_seed(0)
#torch.backends.cudnn.benchmark = False
#torch.use_deterministic_algorithms(F/home/michael/Desktop/backup files/FederatedScope/data/CIKM22Competitionalse)
#import random
#random.seed(0)
#import numpy as np
#np.random.seed(0)

In [7]:
from yacs.config import CfgNode
client_cfg_file = "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_theirs.yaml"# "scripts/B-FHTL_exp_scripts/Graph-DT/cfg_per_client_ours_lr_local_steps.yaml"
client_cfg = CfgNode.load_cfg(open(client_cfg_file,
                                       'r')) if client_cfg_file else None


### Start the FL prosess

In [8]:
from federatedscope.core.auxiliaries.data_builder import get_data
from federatedscope.core.auxiliaries.utils import setup_seed, update_logger
from federatedscope.core.fed_runner import FedRunner
from federatedscope.core.auxiliaries.worker_builder import get_server_cls, get_client_cls


2023-01-07 18:43:37,699 (trainer_builder:11)WARNING: No module named 'federatedscope.contrib.optimizer' in `federatedscope.contrib.trainer`, some modules are not available.


In [9]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_MI_client import LaplacianDomainSeparationMIClient
from federatedscope.contrib.workers.laplacian_server_dom_sep import LaplacianServerDomSep

setup_seed(cfg.seed)
update_logger(cfg)
data, modified_cfg = get_data(cfg)
cfg.merge_from_other_cfg(modified_cfg)


2023-01-07 18:43:37,708 (utils:129)INFO: the current machine is at 127.0.1.1
2023-01-07 18:43:37,709 (utils:131)INFO: the current dir is /home/michael/Master-Thesis/CKIM_Competition
2023-01-07 18:43:37,709 (utils:132)INFO: the output dir is exp/test_dir/Laplacian_MINE_VAE_test_gin_on_graph-dt_lr0.1_lstep1_/sub_exp_20230107184337


In [ ]:
from federatedscope.contrib.workers.laplacian_with_domain_separation_1MINE_VAE_client import LaplacianDomainSeparation1MINEClient
from federatedscope.contrib.workers.laplacian_server import LaplacianServer

Fed_runner = FedRunner(data=data,
                       server_class=LaplacianServer,
                       client_class=LaplacianDomainSeparation1MINEClient,
                       config=cfg.clone(),
                       client_config=client_cfg)
Fed_runner.run()

rho: 0.0
server params: 
encoder_atom.atom_embedding_list.0.weight


/home/michael/Master-Thesis/CKIM_Competition/venv/lib/python3.9/site-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
2023-01-07 18:43:43,787 (fed_runner:249)INFO: Server #0 has been set up ... 
2023-01-07 18:43:43,800 (config:261)INFO: the used configs are: 
asyn:
  min_received_num: 3
  min_received_rate: -1.0
  timeout: 0
  use: True
attack:
  alpha_TV: 0.001
  alpha_prop_loss: 0
  attack_method: 
  attacker_id: -1
  classifier_PIA: randomforest
  info_diff_type: l2
  inject_round: 0
  max_ite: 400
  reconstruct_lr: 0.01
  reconstruct_optim: Adam
  target_label_ind: -1
backend: torch
cfg_file: 
criterion:
  type: MSELoss
data:
  args: []
  batch_size: 64
  cSBM_phi: [0.5, 0.5, 0.5]
  consistent_label_distribution: False
  drop_last: False
  graphsaint:
    num_steps: 30
    walk_length: 2
  loader: 
  num_workers: 0
  pre_transform: []
  quadratic:
    dim: 1
    max_curv: 12.5
  

encoder_atom.atom_embedding_list.1.weight
encoder_atom.atom_embedding_list.2.weight
encoder_atom.atom_embedding_list.3.weight
encoder_atom.atom_embedding_list.4.weight
encoder_atom.atom_embedding_list.5.weight
encoder_atom.atom_embedding_list.6.weight
encoder_atom.atom_embedding_list.7.weight
encoder_atom.atom_embedding_list.8.weight
encoder_atom.atom_embedding_list.9.weight
encoder_atom.atom_embedding_list.10.weight
encoder_atom.atom_embedding_list.11.weight
encoder_atom.atom_embedding_list.12.weight
encoder_atom.atom_embedding_list.13.weight
encoder_atom.atom_embedding_list.14.weight
encoder_atom.atom_embedding_list.15.weight
encoder_atom.atom_embedding_list.16.weight
encoder_atom.atom_embedding_list.17.weight
encoder_atom.atom_embedding_list.18.weight
encoder_atom.atom_embedding_list.19.weight
encoder_atom.atom_embedding_list.20.weight
encoder_atom.atom_embedding_list.21.weight
encoder.weight
encoder.bias
local_gnn.convs.0.nn.linears.0.weight
local_gnn.convs.0.nn.linears.0.bias
loca

2023-01-07 18:43:54,945 (laplacian_with_domain_separation_1MINE_VAE_client:142)INFO: {'Role': 'Client #3', 'Round': 0, 'Results_raw': {'train_total': 3360, 'train_imp_ratio': -221.126394, 'train_avg_loss': 1.795418, 'train_loss': 6032.603909}}


mi_local_global: 0.005899064242839813
mi_global_fixed: -0.0072105154395103455
rec_loss: 0.9160641431808472
kld_loss: 0.11100678890943527
in train
mi_local_global: 0.0001705475151538849
mi_global_fixed: 0.0006589405238628387
rec_loss: 1.012744665145874
kld_loss: 2.6303844451904297
mi_local_global: 0.0007587447762489319
mi_global_fixed: 0.003422800451517105
rec_loss: 0.9957027435302734
kld_loss: 2.2688608169555664
mi_local_global: 0.00019574537873268127
mi_global_fixed: -0.0004719756543636322
rec_loss: 1.0023890733718872
kld_loss: 1.701610803604126
mi_local_global: 0.0020536482334136963
mi_global_fixed: -0.0003128461539745331
rec_loss: 0.9862232208251953
kld_loss: 1.332007646560669
mi_local_global: 0.0015486031770706177
mi_global_fixed: -5.017966032028198e-05
rec_loss: 0.9958133697509766
kld_loss: 1.0581269264221191
mi_local_global: 0.001191418617963791
mi_global_fixed: -0.00040426105260849
rec_loss: 1.000313639640808
kld_loss: 0.898727297782898
mi_local_global: 0.0024278610944747925
mi_

...

mi_local_global: 0.006254151463508606
mi_global_fixed: -0.005296848714351654
rec_loss: 0.9815241098403931
kld_loss: 0.015135284513235092
mi_local_global: 0.006524287164211273
mi_global_fixed: -0.005206409841775894
rec_loss: 0.9647796750068665
kld_loss: 0.01839946210384369
in train
mi_local_global: 0.0009712055325508118
mi_global_fixed: -0.01075781136751175
rec_loss: 0.9862865805625916
kld_loss: 0.6294342875480652
mi_local_global: 0.004190754145383835
mi_global_fixed: -0.010448865592479706
rec_loss: 0.9911472201347351
kld_loss: 0.6118191480636597
mi_local_global: 0.0026953332126140594
mi_global_fixed: -0.011929120868444443
rec_loss: 0.9925954341888428
kld_loss: 0.6103020310401917
mi_local_global: 0.002509918063879013
mi_global_fixed: -0.010792829096317291
rec_loss: 0.9980322122573853
kld_loss: 0.6223381161689758
mi_local_global: 0.000619843602180481
mi_global_fixed: -0.008732397109270096
rec_loss: 0.9825582504272461
kld_loss: 0.6114417314529419
mi_local_global: -0.0001737251877784729
mi

2023-01-07 18:46:12,116 (laplacian_with_domain_separation_1MINE_VAE_client:142)INFO: {'Role': 'Client #1', 'Round': 8, 'Results_raw': {'train_total': 901, 'train_imp_ratio': -393.672456, 'train_avg_loss': 4.563014, 'train_loss': 4111.275948}}


mi_local_global: 0.0029920563101768494
mi_global_fixed: -0.012748487293720245
rec_loss: 0.981045126914978
kld_loss: 0.5332420468330383
in train
mi_local_global: 0.008249785751104355
mi_global_fixed: -0.00835348293185234
rec_loss: 0.9349159598350525
kld_loss: 0.11882620304822922
mi_local_global: 0.008091416209936142
mi_global_fixed: -0.008414745330810547
rec_loss: 0.9309005737304688
kld_loss: 0.10852371156215668
mi_local_global: 0.00877978652715683
mi_global_fixed: -0.00826789066195488
rec_loss: 0.9352043271064758
kld_loss: 0.09626977890729904
mi_local_global: 0.008533172309398651
mi_global_fixed: -0.011539049446582794
rec_loss: 0.928156852722168
kld_loss: 0.10984297096729279
mi_local_global: 0.008885227143764496
mi_global_fixed: -0.01022413745522499
rec_loss: 0.934384286403656
kld_loss: 0.0915079414844513
mi_local_global: 0.007464118301868439
mi_global_fixed: -0.010572172701358795
rec_loss: 0.9220849871635437
kld_loss: 0.09548557549715042
mi_local_global: 0.00831182673573494
mi_global_

2023-01-07 18:46:13,779 (laplacian_with_domain_separation_1MINE_VAE_client:142)INFO: {'Role': 'Client #2', 'Round': 8, 'Results_raw': {'train_total': 512, 'train_imp_ratio': -880.504616, 'train_avg_loss': 0.259834, 'train_loss': 133.034867}}


in train
mi_local_global: 0.02857786789536476
mi_global_fixed: -0.010704029351472855
rec_loss: 0.8080308437347412
kld_loss: 0.013629099354147911
mi_local_global: 0.03028227761387825
mi_global_fixed: -0.008126746863126755
rec_loss: 0.8051942586898804
kld_loss: 0.01656988449394703
mi_local_global: 0.028363201767206192
mi_global_fixed: -0.010769009590148926
rec_loss: 0.8057382702827454
kld_loss: 0.015577616170048714
mi_local_global: 0.029687467962503433
mi_global_fixed: -0.010954014956951141
rec_loss: 0.8002978563308716
kld_loss: 0.016746286302804947
mi_local_global: 0.02963675931096077
mi_global_fixed: -0.010140381753444672
rec_loss: 0.8062647581100464
kld_loss: 0.017112279310822487
mi_local_global: 0.02872346341609955
mi_global_fixed: -0.010005272924900055
rec_loss: 0.807961642742157
kld_loss: 0.013781685382127762
mi_local_global: 0.028051290661096573
mi_global_fixed: -0.01278487965464592
rec_loss: 0.8071262836456299
kld_loss: 0.014664540067315102
mi_local_global: 0.028840046375989914
m

client 0 times: 0.346s
bei normal fedavg: 0.15s
mit MINE: 1.7 Sekunden


+

In [ ]:
777-431


In [ ]:
data

In [ ]:
import os

,,0After 10 rounds:

diff: 0, csd: 0:
server: -8
c1 t/val: 0.375 /0.37
c2 t/val: 0.678 /0.606
c3 t/val: 0.699 /0.608
c4 t/val: 0.599 /0.537

diff: 0, csd: 0:
server: -4.5
c1 t/val: 0.372879 /0.328
c2 t/val: 0.678992 /0.605809
c3 t/val: 0.70166 /0.608609
c4 t/val: 0.597455 /0.534253



diff: 10, csd: 0:
server: -35
c1 t/val: 0.482/0.52
c2 t/val: 0.694/0.65
c3 t/val: 0.724/0.65
c4 t/val: 0.674/0.575

diff: 1, csd: 0:
server: -9.6
c1 t/val: 0.378194/0.319349
c2 t/val: 0.67461/0.609163
c3 t/val: 0.705559/0.615019
c4 t/val: 0.605581/0.572086

diff: 0, csd: 1e4:
server: -18
c1 t/val: 0.280962/0.23074
c2 t/val: 0.682652/0.603175
c3 t/val: 0.699094/0.67027
c4 t/val: 0.594985/0.609611

diff: 0 csd: 1e8
server: -18
c1 t/val: 0.2606/0.348(0.197)
c2 t/val: 0.653/0.0.602
c3 t/val: 0.691/0.597
c4 t/val: 0.581/0.629

diff: 0 csd: 1e8
server: -12.7
c1 t/val: 0.262647/0.249875
c2 t/val: 0.649273/0.0.600435
c3 t/val: 0.696132/0.615158
c4 t/val: 0.586597/0.656992


diff: 0.1 csd: 0
server: -11
c1 t/val: 0.2606/0.332837
c2 t/val: 0.678326/0.0.610099
c3 t/val: 0.704083/0.616179
c4 t/val: 0.604177/0.559658




In [ ]:
os.environ['CUBLAS_WORKSPACE_CONFIG']